In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
import itertools
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from ExperimentTracker2 import PhaseOneExperimentTracker
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from dask_ml.model_selection import train_test_split

# Initialize Dask client for parallel processing
client = Client()

In [ ]:
# Load large dataset with Dask
df = dd.read_csv("merged_data.csv")

In [ ]:
# Convert data types for memory efficiency
def convert_dtypes(df):
    df['order_value'] = df['order_value'].astype('float32')
    df['refund_value'] = df['refund_value'].astype('float32')
    df['num_items_ordered'] = df['num_items_ordered'].astype(float).round().astype('uint8')
    df['order_date'] = dd.to_datetime(df['order_date'])
    df['first_order_datetime'] = dd.to_datetime(df['first_order_datetime'])
    df[['country_code', 'collect_type', 'payment_method']] = df[['country_code', 'collect_type', 'payment_method']].astype('category')
    return df

df = convert_dtypes(df)

In [ ]:
# Payment method grouping
def group_payment_methods(payment_method):
    mapping = {
        'CreditCard': ['GenericCreditCard', 'CybersourceCreditCard', 'CybersourceApplePay', 'CreditCard'],
        'DigitalWallet': ['GCash', 'AFbKash', 'JazzCashWallet', 'AdyenBoost', 'PayPal'],
        'BankTransfer': ['XenditDirectDebit', 'RazerOnlineBanking'],
        'PaymentOnDelivery': ['Invoice', 'PayOnDelivery']
    }
    for key, values in mapping.items():
        if payment_method in values:
            return key
    return 'Others'

df['payment_method'] = df['payment_method'].map(group_payment_methods)

In [ ]:
# Date transformations
def date_transformations(df):
    df['days_since_first_order'] = (df['order_date'] - df['first_order_datetime']).dt.days
    df = df.drop(columns=['first_order_datetime'])
    df['order_date_day_of_week'] = df['order_date'].dt.dayofweek
    df['order_date_day'] = df['order_date'].dt.day
    df['order_date_month'] = df['order_date'].dt.month
    df['order_date_year'] = df['order_date'].dt.year
    df = df.drop(columns=['order_date'])
    return df

df = date_transformations(df)
df = df.drop(columns=['order_id', 'customer_id'])

In [ ]:
# Split data
X = df.drop(columns=['is_fraud'])
y = df['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Define experiment configurations
search_space = {
    'scaler': [None, StandardScaler(), MinMaxScaler(), RobustScaler()],
    'encode': [{'apply': True, 'columns': ['categorical_col']}],
    'models': [
        {'name': 'LogisticRegression', 'instance': LogisticRegression()},
        {'name': 'RandomForest', 'instance': RandomForestClassifier()},
        {'name': 'LightGBM', 'instance': LGBMClassifier()},
        {'name': 'GaussianNB', 'instance': GaussianNB()},
        {'name': 'DecisionTree', 'instance': DecisionTreeClassifier()},
        {'name': 'GradientBoosting', 'instance': GradientBoostingClassifier()},
    ]
}

# Generate all combinations
keys, values = zip(*search_space.items())
experiment_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

categorical_cols = ['payment_method', 'country_code', 'collect_type']
numeric_columns = ['order_value', 'refund_value', 'num_items_ordered', 'days_since_first_order',
                   'order_date_day_of_week', 'order_date_day', 'order_date_month', 'order_date_year']

In [ ]:
print(len(experiment_combinations))

In [ ]:
print(experiment_combinations,X_train.shape,y_train.shape,X_test.shape,y_test.shape,numeric_columns,categorical_cols)

In [ ]:
from ExperimentTracker2 import PhaseOneExperimentTracker

In [ ]:
# client = Client(local_directory="dask_tmp")

# X_train_future = client.scatter(X_train, broadcast=True)
# y_train_future = client.scatter(y_train, broadcast=True)
# X_test_future = client.scatter(X_test, broadcast=True)
# y_test_future = client.scatter(y_test, broadcast=True)

In [ ]:
# # Initialize tracker and run experiments
# tracker = PhaseOneExperimentTracker("Phase-1 (Final)")
# tracker.run_experiments(experiment_combinations, X_train, y_train, X_test, y_test, numeric_columns, categorical_cols)

---

In [ ]:
# Load datasets
X_train = dd.read_csv('./data/X_train.csv')
y_train = dd.read_csv('./data/y_train.csv')

X_test = dd.read_csv('./data/X_test.csv')
y_test = dd.read_csv('./data/y_test.csv')

X_train_ISO = dd.read_csv('./data/X_train_ISO.csv')
y_train_ISO = dd.read_csv('./data/y_train_ISO.csv')

X_train_ISO_SMOTE = dd.read_csv('./data/X_train_ISO_smote.csv')
y_train_ISO_SMOTE = dd.read_csv('./data/y_train_ISO_smote.csv')

X_train_ISO_ROS = dd.read_csv('./data/X_train_ISO_ros.csv')
y_train_ISO_ROS = dd.read_csv('./data/y_train_ISO_ros.csv')

X_train_ISO_RUS = dd.read_csv('./data/X_train_ISO_rus.csv')
y_train_ISO_RUS = dd.read_csv('./data/y_train_ISO_rus.csv')

X_train_LOF = dd.read_csv('./data/X_train_LOF.csv')
y_train_LOF = dd.read_csv('./data/y_train_LOF.csv')

X_train_LOF_SMOTE = dd.read_csv('./data/X_train_LOF_smote.csv')
y_train_LOF_SMOTE = dd.read_csv('./data/y_train_LOF_smote.csv')

X_train_LOF_ROS = dd.read_csv('./data/X_train_LOF_ros.csv')
y_train_LOF_ROS = dd.read_csv('./data/y_train_LOF_ros.csv')

X_train_LOF_RUS = dd.read_csv('./data/X_train_LOF_rus.csv')
y_train_LOF_RUS = dd.read_csv('./data/y_train_LOF_rus.csv')

X_train_smote = dd.read_csv('./data/X_train_smote.csv')
y_train_smote = dd.read_csv('./data/y_train_smote.csv')

X_train_ros = dd.read_csv('./data/X_train_ros.csv')
y_train_ros = dd.read_csv('./data/y_train_ros.csv')

X_train_rus = dd.read_csv('./data/X_train_rus.csv')
y_train_rus = dd.read_csv('./data/y_train_rus.csv')

In [ ]:
datasets = [
    ("dataset_default", X_train, y_train),
    ("dataset_ISO", X_train_ISO, y_train_ISO),
    ("dataset_ISO_SMOTE", X_train_ISO_SMOTE, y_train_ISO_SMOTE),
    ("dataset_ISO_ROS", X_train_ISO_ROS, y_train_ISO_ROS),
    ("dataset_ISO_RUS", X_train_ISO_RUS, y_train_ISO_RUS),
    ("dataset_LOF", X_train_LOF, y_train_LOF),
    ("dataset_LOF_SMOTE", X_train_LOF_SMOTE, y_train_LOF_SMOTE),
    ("dataset_LOF_ROS", X_train_LOF_ROS, y_train_LOF_ROS),
    ("dataset_LOF_RUS", X_train_LOF_RUS, y_train_LOF_RUS),
    ("dataset_SMOTE", X_train_smote, y_train_smote),
    ("dataset_ROS", X_train_ros, y_train_ros),
    ("dataset_RUS", X_train_rus, y_train_rus)
]

In [ ]:
# Define experiment configurations
search_space = {
    'scaler': [None],
    'encode': [{'apply': True, 'columns': ['categorical_col']}],
    'models': [
        {'name': 'LogisticRegression', 'instance': LogisticRegression()},
        {'name': 'RandomForest', 'instance': RandomForestClassifier()},
        {'name': 'LightGBM', 'instance': LGBMClassifier()},
        {'name': 'GaussianNB', 'instance': GaussianNB()},
        {'name': 'DecisionTree', 'instance': DecisionTreeClassifier()},
        {'name': 'GradientBoosting', 'instance': GradientBoostingClassifier()},
    ]
}

# Generate all combinations
keys, values = zip(*search_space.items())
experiment_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

categorical_cols = ['payment_method', 'country_code', 'collect_type']
numeric_columns = ['order_value', 'refund_value', 'num_items_ordered', 'days_since_first_order',
                   'order_date_day_of_week', 'order_date_day', 'order_date_month', 'order_date_year']

In [ ]:
len(experiment_combinations)

In [ ]:
# # Initialize the tracker
# from ExperimentTracker2 import PhaseTwoExperimentTracker
# tracker = PhaseTwoExperimentTracker("Phase-2 (Final)")

# # Load checkpoint file
# tracker.completed_runs

# # Run experiments with checkpointing
# tracker.run_experiments(
#     datasets=datasets,
#     experiment_combinations=experiment_combinations,
#     X_test=X_test,
#     y_test=y_test,
#     numeric_columns=numeric_columns,
#     categorical_cols=categorical_cols
# )

In [ ]:
defined_experiment_combinations = [
    {
        "scaler": MinMaxScaler(),
        "encode": {"apply": True, "columns": ["categorical_col"]},
        "models": {"name": "RandomForest", "instance": RandomForestClassifier()}
    },
    {
        "scaler": None,
        "encode": {"apply": True, "columns": ["categorical_col"]},
        "models": {"name": "RandomForest", "instance": RandomForestClassifier()}
    },
    {
        "scaler": StandardScaler(),
        "encode": {"apply": True, "columns": ["categorical_col"]},
        "models": {"name": "LightGBM", "instance": LGBMClassifier()}
    },
    {
        "scaler": None,
        "encode": {"apply": True, "columns": ["categorical_col"]},
        "models": {"name": "LightGBM", "instance": LGBMClassifier()}
    },
]

In [ ]:
from ExperimentTrackers2 import PhaseThreeExperimentTracker

tracker = PhaseThreeExperimentTracker("Models with Scaler")

# Load checkpoint file
tracker.completed_runs

# Run experiments with checkpointing
tracker.run_experiments(
    datasets=datasets,
    experiment_combinations=defined_experiment_combinations,
    X_test=X_test,
    y_test=y_test,
    numeric_columns=numeric_columns,
    categorical_cols=categorical_cols
)
